# Python Code for calculating Ampacity of Transmission Line using IEEE 738 Standards

In [ ]:
import numpy as np
import pandas as pd

# ==============================
# Step 1. Conductor Parameters
# ==============================

# ACSR Drake (typical transmission conductor)

D = 0.0281               # Conductor diameter (m)
R20 = 0.0214             # Resistance at 20C (ohm/km)
alpha_R = 0.004          # Temperature coefficient

epsilon = 0.5            # Emissivity
alpha_s = 0.5            # Solar absorptivity

# Convert resistance to ohm/m
R20 = R20 / 1000

T_max = 75.0             # Max allowable current temperature (C)

# ==============================
# Step 2. Weather Data (supposed)
# ==============================

data = {
    "Ta" : [5, 10, 15, 40],         # Ambient Temperature (C)
    "Vw" : [1, 2.5, 2.0, 0.6],      # Wind Speed (m/s)
    "theta" : [22.5, 30, 45, 90],   # Wind angle (degrees)
    "S" : [100, 500, 800, 1000]      # Solar radiation (W/m^2)
}
df=pd.DataFrame(data)

# ==============================
# Step 3. IEEE 738 Functions
# ==============================

#perpendicular wind component

def effective_wind_speed(Vw, theta_deg):
    return Vw * np.sin(np.radians(theta_deg))

 #IEEE 738 forced and natural convection

def convective_cooling(Tc, Ta, Veff):

    delta_T = Tc - Ta

# Natural Convection
    Qc_nat = 3.645 * (D ** 0.75) * (delta_T ** 1.25)

# forced convection
    Qc_for = 0.754 * (D ** 0.6) * (Veff ** 0.6) * delta_T

    return max(Qc_nat, Qc_for)

def radiative_cooling(Tc, Ta):
    return 17.8 * D * epsilon * (((Tc+ 273)/100) ** 4 -((Ta+273)/100) ** 4)

# Solar heat Gain (W/m)
def solar_heating(S):
    return alpha_s * S * D

# Temperature- dependent resistance
def resistance_at_temp(Tc):
    return R20 * (1+ alpha_R * (Tc - 20))

#Solve IEEE 738 Steady-state ampacity
def dynamic_ampacity(Ta, Vw, theta, S):

    Veff = effective_wind_speed(Vw, theta)
    # Veff = max(Veff, 0.01)


    Qc = convective_cooling(T_max, Ta, Veff)
    Qr= radiative_cooling(T_max, Ta)
    Qs = solar_heating(S)

    R_T = resistance_at_temp(T_max)

    numerator = Qc + Qr - Qs

    if numerator <= 0:
        return 0

    I= np.sqrt(numerator / R_T)
    return I

# =================================
# Step 4. Compute Dynamic Ampacity
# =================================

df["Dynamic_Ampacity_A"] = df.apply(
    lambda row: dynamic_ampacity(
        row["Ta"], row["Vw"], row["theta"], row["S"]
    ),
    axis=1
)

# ==============================
# Step 3. Print Result
# ==============================
print(df)

# Static Ampacity (A conservative Approach)

In [ ]:
# ==========================
# Static Rating Assumptions : Widely used values in US
# ==========================

Ta_static = 40.0        # Ambient Temperature (C)
Vw_static = 0.6         # Wind Speed (m/s)
theta_static = 90.0     # Wind Perpendicular (degree)
S_static = 1000.0       # Solar radiation ( W/m^2)

# ==========================
# Static Ampacity
# ==========================

I_static = dynamic_ampacity(
    Ta_static,
    Vw_static,
    theta_static,
    S_static
)

print(f"Static Rating Ampacity: {I_static:.1f} A")

# Compare Static Vs Dynamic Rating (Hourly)

In [ ]:
df["Static_Ampacity_A"] = I_static

df["Ampacity_Increase_A"] = (
    df["Dynamic_Ampacity_A"] - df["Static_Ampacity_A"]
)

df["Ampacity_Increase_pct"] = (
    df["Ampacity_Increase_A"] / df["Static_Ampacity_A"] *100
)

# Making Time Explicit, Adding an hour column

In [ ]:
df["Hour"] = np.arange(1, len(df) + 1)

# Output Table

In [ ]:
output = df[[
    "Hour",
    "Static_Ampacity_A",
    "Dynamic_Ampacity_A",
    "Ampacity_Increase_pct"
]].copy()

# Round for presentation
output["Static_Ampacity_A"] = output["Static_Ampacity_A"].round(0)
output["Dynamic_Ampacity_A"] = output["Dynamic_Ampacity_A"].round(0)
output["Ampacity_Increase_pct"] = output["Ampacity_Increase_pct"].round(1)

print(output.to_string(index=False))


In [ ]:
output.columns = [
    "Hour",
    "Static Rating (A)",
    "Dynamic Rating (A)",
    "Increase (%)"
]

print(output)


# Result Graphs between Dynamic and Static Ampacity

In [ ]:
import matplotlib.pyplot as plt

# ==============================
# PLOT 1: Static vs Dynamic Ampacity
# ==============================

plt.figure()
plt.plot(df["Hour"], df["Dynamic_Ampacity_A"], label="Dynamic Ampacity")
plt.plot(df["Hour"], df["Static_Ampacity_A"], label="Static Ampacity")

plt.xlabel("Hour")
plt.ylabel("Ampacity (A)")
plt.title("Static vs Dynamic Line Ampacity")
plt.legend()
plt.grid(True)

plt.show()

# ==============================
# PLOT 2: Ampacity Change/ Increase (%)
# ==============================

plt.plot(df["Hour"], df["Ampacity_Increase_pct"])

plt.xlabel("Hour")
plt.ylabel("Ampacity Increase (%)")
plt.title("Hourly Ampacity Changes Relative to Static Rating")
plt.grid(True)

plt.show()
